In [ ]:
#CONTENT BASED MODEL
#Carichiamo i dataframe di test e train
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow_recommenders as tfrs
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
import numpy as np

df_train = pd.read_csv('rankFM_training.csv')
df_test = pd.read_csv('rankFM_test.csv')
df_val = pd.read_csv('rankFM_validation.csv')

#print(len(df_train['person_id'].unique()))
#print(len(df_val['person_id'].unique()))
#print(len(df_test['person_id'].unique()))
#Find the interactions with new books and user (merges)
#Only users in 2_users.csv and books in 2_final_dataset.csv
df_users = pd.read_csv('2_users.csv')
df_books = pd.read_csv('2_final_dataset.csv')
df_users['person_id'] = df_users['person_id'].astype(str)
df_books['title'] = df_books['title'].astype(str)

#VOCABULARY CREATION
vocabulary_titles = np.asarray(df_books['title'].unique().tolist()) #LIST OF TITLES
vocabulary_users = np.asarray(df_users['person_id'].unique().tolist()) #LIST OF USERS
num_books = len(vocabulary_titles) #NUM BOOKS
num_users = len(vocabulary_users) #NUM USERS

df_train['rating'] = 1
df_val['rating'] = 1
df_test['rating'] = 1

df_train = df_train.merge(df_users, on='person_id')
df_train = df_train.drop(df_train.columns[[0] + [4]], axis = 1)
df_test = df_test.merge(df_users, on='person_id')
df_test = df_test.drop(df_test.columns[[0] + [4]], axis = 1)
df_val = df_val.merge(df_users, on='person_id')
df_val = df_val.drop(df_val.columns[[0] + [4]], axis = 1)

df_train = df_train.merge(df_books, on='book_id')
#df_train = df_train.drop(df_train.columns[[i for i in range(5, 20)] + [i for i in range(21, df_train.shape[1])]], axis = 1)
df_test = df_test.merge(df_books, on='book_id')
#df_test = df_test.drop(df_test.columns[[i for i in range(5, 20)] + [i for i in range(21, df_test.shape[1])]], axis = 1)
df_val = df_val.merge(df_books, on='book_id')
#df_val = df_val.drop(df_val.columns[[i for i in range(5, 20)] + [i for i in range(21, df_val.shape[1])]], axis = 1)


#ONLY USERS IN ALL 3 DATASETS
list_users = pd.DataFrame({'person_id': df_train['person_id'].unique()})
df_test = df_test.merge(list_users, on='person_id')
df_val = df_val.merge(list_users, on='person_id')

#OUTPUT CELL: RATINGS AT 1 (IMPLICIT) WITH FEATURES

In [ ]:
#MATRIX SIMILARITY CREATION
#Let's represent genres as a one hot encoding in different fields of the dataframe.
#GENRES STARTING AT FIELD 30
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens') #For text embedding

mapping = {'Comics&GraphicNovels': 0,
              'Family-Sex&Relationships': 1,
              'Humor': 2,
              'History': 3,
              'ScienceFiction&Fantasy': 4,
              'Romance': 5,
              'Travel': 6,
              'Mystery&Thrillers': 7,
              'FreeTime': 8,
              'Non-fiction': 9,
              'Biography': 10,
              'SocialScience': 11,
              'Political': 12,
              'Crime': 13,
              'Children&Teens': 14,
              'Philosophy': 15,
              'Horror': 16,
              'Health-Mind&Body': 17,
              'Professional&Technical': 18,
              'Science&Nature': 19}
              #'Fiction&Literature': 20}
    
DFbooks_integrated_without = df_books
DFbooks_integrated_without = DFbooks_integrated_without.drop(DFbooks_integrated_without.columns[[0,1]], axis=1)
#display(DFbooks_integrated_without.dtypes)
#Functions to add fields
def genre_vector_function(x, genre):
    #Calculate total rating to normalize
    dict_ = eval(x['genre'])
    total_scores = 0
    for key in dict_:
        total_scores += float(dict_[key])
    if genre in list(eval(x['genre']).keys()):
        return float(dict_[genre]) / total_scores
    else:
        return 0

def mood_vector_function(x, mood):
    mood_book = x['majority']
    if mood_book == mood:
        return 1.0
    else:
        return 0

def sentiment_vector_function(x, sentiment):
    sentiment_book = x['emotion']
    #print(sentiment_book)
    if sentiment_book == sentiment:
        return 1.0
    else:
        return 0

def description_vector_function(x):
    #For this, we will use BERTvectorizer to transform an unstructured text into a numeric vector
    vector_description = model.encode(x['content2'])
    return vector_description

def title_vector_function(x):
    #For this, we will use BERTvectorizer to transform an unstructured text into a numeric vector
    title_description = model.encode(x['title'])
    return title_description

def keyword_vector_function(x, flag):
    #print(x)
    #Calculate total rating to normalize
    if flag == 0:
        str_keywords = x['top_keywords_5_with_comment']
    elif flag == 1:
        str_keywords = x['top_keywords_10_with_comment']
    elif flag == 2:
        str_keywords = x['top_keywords_15_with_comment']
    elif flag == 3:
        str_keywords = x['top_keywords_20_with_comment']
    elif flag == 4:
        str_keywords = x['top_keywords_5_without_comm']
    elif flag == 5:
        str_keywords = x['top_keywords_10_without_comm']
    elif flag == 6:
        str_keywords = x['top_keywords_15_without_comm']
    elif flag == 7:
        str_keywords = x['top_keywords_20_without_comm']
    vector_keywords = model.encode(str_keywords)
    return vector_keywords
    
#Let's calculate the onehot encoding of features (genres, moods, sentiments, description, keywords as a text vector)
DFbooks_integrated_without_genre_vector = DFbooks_integrated_without

for genre in list(mapping.keys()):
    DFbooks_integrated_without_genre_vector[genre] = DFbooks_integrated_without_genre_vector.apply(genre_vector_function, genre=genre, axis = 1)

for mood in ['positive', 'negative']:
    DFbooks_integrated_without_genre_vector[mood] = DFbooks_integrated_without_genre_vector.apply(mood_vector_function, mood=mood, axis = 1)

for sentiment in ['sadness', 'joy', 'fear', 'anger']:
    DFbooks_integrated_without_genre_vector[sentiment] = DFbooks_integrated_without_genre_vector.apply(sentiment_vector_function, sentiment=sentiment, axis = 1)
    
#DFbooks_integrated_without_genre_vector['vector_keywords_5_with'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=0, axis = 1)
DFbooks_integrated_without_genre_vector['vector_keywords_10_with'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=1, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_15_with'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=2, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_20_with'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=3, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_5_without'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=4, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_10_without'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=5, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_15_without'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=6, axis = 1)
#DFbooks_integrated_without_genre_vector['vector_keywords_20_without'] = DFbooks_integrated_without_genre_vector.apply(keyword_vector_function,flag=7, axis = 1)


DFbooks_integrated_without_genre_vector['vector_title'] = DFbooks_integrated_without_genre_vector.apply(title_vector_function, axis = 1)

DFbooks_integrated_without_genre_vector['vector_description'] = DFbooks_integrated_without_genre_vector.apply(description_vector_function, axis = 1)

In [ ]:
#MATRIX BOOKS SIMILARITY CONTENT BASED
import numpy as np

#Description matrix similarity
list_vector_descriptions = np.array(DFbooks_integrated_without_genre_vector['vector_description'].to_list())
list_vector_keywords = np.array(DFbooks_integrated_without_genre_vector['vector_keywords_20_without'].to_list())
list_vector_titles = np.array(DFbooks_integrated_without_genre_vector['vector_title'].to_list())

description_similarity = list_vector_descriptions @ list_vector_descriptions.T
keywords_similarity = list_vector_keywords @ list_vector_keywords.T
title_similarity = list_vector_titles @ list_vector_titles.T

#Calculate vector and dot product
book_vector = np.array(DFbooks_integrated_without_genre_vector.iloc[:,35:-3]) #CHANGE INDEX TO CHANGE CONSIDERED FEATURES
dot_product = book_vector.dot(np.transpose(book_vector))
dot_product = dot_product + title_similarity + description_similarity + keywords_similarity #Description title  keywords influence

#CONTENT BASED RECOMMENDATION FOR USERS 

In [ ]:
import random
#USER LIST
list_users = df_train['person_id'].unique().tolist()

#TO EVALUATE AVERAGE HIT USERS AND AVERAGE HIT RECOMMENDATIONS
users = []
recomms = []
user_genre_counter = {}
for index, user in enumerate(list_users):
    if index % 10000 == 0:
        print(index)
    #For evaluation
    user_flag = 0
    recomms_count = 0
    
    #Extract user interactions
    DFuser_interactions = df_train[df_train['person_id'] == user]

    #Extract read books
    user_read_books = df_test[df_test['person_id'] == user].title.to_list()
    #user_read_books_train = df_train[df_train['person_id'] == user].title.to_list()
    
    if user_read_books == []:
        continue
        
    #Retrieve book IDs
    list_book_IDs = DFuser_interactions['title'].to_list()

    #Let's suggest 20 recommendations
    dict_recomm = {}
    suggested_titles = []
    
    #From book ID, retrieve index for matrix similarity
    #Let's choose a random read book of the user
    identifiers = random.choices(list_book_IDs, k=20)
    
    for identifier in identifiers:
        if identifier not in dict_recomm.keys():
            dict_recomm[identifier] = 2
        else:
            dict_recomm[identifier] += 1

        #Index retrieved
        DFloc = DFbooks_integrated_without_genre_vector.loc[DFbooks_integrated_without_genre_vector.title == identifier].index.to_list()[0]

        #Find the highest value by fixing the row at index
        row = dot_product[DFloc, :]
        index_book = row.argsort()[-dict_recomm[identifier]]

        #Find book
        title_2 = DFbooks_integrated_without_genre_vector.loc[index_book].title
        genres_2 = DFbooks_integrated_without_genre_vector.loc[index_book].genre
             
        if title_2 in user_read_books:
            recomms_count += 1
            user_flag = 1
            
            #COUNT GENRES FOR STATISTICS
            dict_genre = eval(genres_2)
            for genre in dict_genre.keys():
                if genre in user_genre_counter.keys():
                    user_genre_counter[genre] += 1
                else:
                    user_genre_counter[genre] = 1
                    
    recomms.append(recomms_count)
    users.append(user_flag)
    #print(users)
mean_users = np.mean(np.asarray(users))
mean_recomms = np.mean(np.asarray(recomms))
print(mean_users)
print(mean_recomms)
print(user_genre_counter)